#### Para extrair novos dados utilizaremos o mesmo codigo que utilizamos para obter a base de treinos inicialmente, porem com o link alterado para exibir apenas vagas postadas no linkedin nas ultimas 24h

In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configurar o WebDriver do Selenium
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)

# URL do site para extrair os dados
url = "https://www.linkedin.com/jobs/search?keywords=%28%22%28data%20Science%29%22%20OR%20%22cientista%20De%20Dados%29%22%20OR%20%22machine%20Learning%22%29&location=Brasil&locationId=&geoId=106057199&f_TPR=r86400&position=1&pageNum=0"
# Lista para armazenar os dados extraídos
data = []

# Acessar a página
driver.get(url)
wait = WebDriverWait(driver, 10)

# Tempo de espera para carregar mais trabalhos (ajuste conforme necessário)
wait_time = 3


# Rolar a página até o final máximo possível
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(wait_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
# Verificar se há o botão "Ver mais vagas" e clicar nele até 30 vezes, se necessário
click_count = 0
max_clicks = 10
while click_count < max_clicks:
    show_more_jobs_button = driver.find_element(By.CSS_SELECTOR, ".infinite-scroller__show-more-button")
    if show_more_jobs_button.is_displayed():
        show_more_jobs_button.click()
        time.sleep(wait_time)
        click_count += 1
        print(click_count)
    else:
        break

# Extrair os elementos de trabalho da página
soup = BeautifulSoup(driver.page_source, "html.parser")
job_elements = soup.select(".base-card__full-link")

# Extrair os dados de cada trabalho
for element in job_elements:
    # Verificar se o elemento possui o atributo href
    if "href" not in element.attrs:
        continue

    # Obter o link do trabalho
    link = element["href"]

    # Clicar no elemento para abrir a vaga na página dinâmica
    driver.get(link)
    time.sleep(wait_time)

    # Extrair o título do trabalho
    title_element = driver.find_element(By.CSS_SELECTOR, ".top-card-layout__title")
    if title_element is not None:
        title = title_element.text.strip()
    else:
        title = ""

    # Extrair o nome da empresa
    company_element = driver.find_element(By.CSS_SELECTOR, ".topcard__org-name-link")
    if company_element is not None:
        company = company_element.text.strip()
    else:
        company = ""

    # Extrair a localização
    location_element = driver.find_element(By.CSS_SELECTOR, ".topcard__flavor--bullet")
    if location_element is not None:
        location = location_element.text.strip()
    else:
        location = ""

    # Clicar no botão "Exibir mais" para obter a descrição expandida
    show_more_button = driver.find_element(By.CSS_SELECTOR, ".show-more-less-html__button")
    if show_more_button is not None:
        show_more_button.click()
        time.sleep(wait_time)

        # Extrair a descrição expandida
        expanded_description_element = driver.find_element(By.CSS_SELECTOR, ".show-more-less-html__markup")
        if expanded_description_element is not None:
            expanded_description = expanded_description_element.text.strip()
        else:
            expanded_description = ""
    else:
        expanded_description = ""

    # Verificar se o trabalho já está na lista de dados
    job_id = link.split("/")[-1].split("?")[0]
    if any(job_id == item["ID"] for item in data):
        continue

    # Adicionar os dados à lista
    data.append({
        "ID": job_id,
        "DATA": pd.Timestamp.now().strftime("%Y-%m-%d"),
        "TITULO": title,
        "EMPRESA": company,
        "LOCAL": location,
        "LINK": link,
        "DESCRICAO": expanded_description
    })

    # Converter a lista de dados em um DataFrame do Pandas
    df = pd.DataFrame(data)
    
    # Exibir uma amostra do DataFrame
    print(df.sample())

    # Salvar os dados em um arquivo CSV
    df.to_csv("newly_fetched_data.csv", index=False)

# Exibir uma amostra do DataFrame
print(df.sample(5))

# Fechar o WebDriver
driver.quit()


                                                  ID        DATA  \
0  cientista-de-dados-sr-pol%C3%ADticas-de-cr%C3%...  2023-07-13   

                                         TITULO EMPRESA          LOCAL  \
0  Cientista de Dados Sr - Políticas de Crédito   Porto  São Paulo, SP   

                                                LINK  \
0  https://br.linkedin.com/jobs/view/cientista-de...   

                                           DESCRICAO  
0  Job Description\n\nA área de Riscos e Política...  
                                                  ID        DATA  \
0  cientista-de-dados-sr-pol%C3%ADticas-de-cr%C3%...  2023-07-13   

                                         TITULO EMPRESA          LOCAL  \
0  Cientista de Dados Sr - Políticas de Crédito   Porto  São Paulo, SP   

                                                LINK  \
0  https://br.linkedin.com/jobs/view/cientista-de...   

                                           DESCRICAO  
0  Job Description\n\nA área de Risco

                                                  ID        DATA  \
4  cientista-de-dados-s%C3%AAnior-at-innolevels-3...  2023-07-13   

                      TITULO     EMPRESA               LOCAL  \
4  Cientista de dados Sênior  innolevels  Rio de Janeiro, RJ   

                                                LINK  \
4  https://br.linkedin.com/jobs/view/cientista-de...   

                                           DESCRICAO  
4  Especialista em Data Science e Analytics, tend...  
                                                  ID        DATA  \
2  cientista-de-dados-s%C3%AAnior-home-office-at-...  2023-07-13   

                                    TITULO  EMPRESA               LOCAL  \
2  Cientista de dados sênior (Home Office)  Farisco  São Paulo e Região   

                                                LINK  \
2  https://br.linkedin.com/jobs/view/cientista-de...   

                                           DESCRICAO  
2  Irá atuar com extração e manipulação de grande...  


                                                   ID        DATA  \
12  consultor-de-supply-chain-at-accenture-brasil-...  2023-07-13   

                       TITULO           EMPRESA          LOCAL  \
12  Consultor de Supply Chain  Accenture Brasil  São Paulo, SP   

                                                 LINK  \
12  https://br.linkedin.com/jobs/view/consultor-de...   

                                            DESCRICAO  
12  Sobre a Accenture. A Accenture é uma empresa g...  
                                             ID        DATA  \
31  cientista-de-dados-at-innolevels-3661999476  2023-07-13   

                TITULO     EMPRESA               LOCAL  \
31  Cientista de Dados  innolevels  Rio de Janeiro, RJ   

                                                 LINK  \
31  https://br.linkedin.com/jobs/view/cientista-de...   

                                            DESCRICAO  
31  Buscar impacto positivo para nossos clientes a...  
                              

                                                 ID        DATA  \
34  analista-de-dados-at-grupo-sh-brasil-3663173271  2023-07-13   

               TITULO          EMPRESA         LOCAL  \
34  Analista de Dados  GRUPO SH BRASIL  Salvador, BA   

                                                 LINK  \
34  https://br.linkedin.com/jobs/view/analista-de-...   

                                            DESCRICAO  
34  O Grupo SH Brasil tem como principal objetivo ...  
                                         ID        DATA             TITULO  \
25  banco-de-talentos-at-certsys-3663279322  2023-07-13  Banco de Talentos   

    EMPRESA          LOCAL                                               LINK  \
25  Certsys  São Paulo, SP  https://br.linkedin.com/jobs/view/banco-de-tal...   

                                            DESCRICAO  
25  Job Description\n\nCom um histórico de 16 anos...  
                                                   ID        DATA  \
46  senior-python-backen

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008CA813+48355]
	(No symbol) [0x0085C4B1]
	(No symbol) [0x00765358]
	(No symbol) [0x00761B33]
	(No symbol) [0x00758CA5]
	(No symbol) [0x00759D0A]
	(No symbol) [0x00759003]
	(No symbol) [0x00758355]
	(No symbol) [0x007582EC]
	(No symbol) [0x00756F26]
	(No symbol) [0x00757688]
	(No symbol) [0x00766B52]
	(No symbol) [0x007BCF01]
	(No symbol) [0x007AA73C]
	(No symbol) [0x007BC922]
	(No symbol) [0x007AA536]
	(No symbol) [0x007882DC]
	(No symbol) [0x007893DD]
	GetHandleVerifier [0x00B2AABD+2539405]
	GetHandleVerifier [0x00B6A78F+2800735]
	GetHandleVerifier [0x00B6456C+2775612]
	GetHandleVerifier [0x009551E0+616112]
	(No symbol) [0x00865F8C]
	(No symbol) [0x00862328]
	(No symbol) [0x0086240B]
	(No symbol) [0x00854FF7]
	BaseThreadInitThunk [0x75FA7D59+25]
	RtlInitializeExceptionChain [0x7728B79B+107]
	RtlClearBits [0x7728B71F+191]


In [3]:
len(df)

65